# Visualisation de l'Entraînement du Modèle Hunyuan3D pour les Lunettes

Ce notebook permet de visualiser les résultats de l'entraînement du modèle adapté de Hunyuan3D 2.0 pour la génération de lunettes 3D.

In [ ]:
import os
import sys
import json
import glob
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch
import trimesh
from tqdm.notebook import tqdm

# Ajout du répertoire parent au path pour importer les modules personnalisés
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))
from models.hunyuan3d_adapted import Hunyuan3DGlassesAdapter, GlassesGenerator

## 1. Configuration

In [ ]:
# Répertoire des modèles entraînés
MODELS_DIR = "../models/saved"

# Répertoire des données de test
TEST_DATA_DIR = "../data/test"

# Vérification de l'existence des répertoires
if not os.path.exists(MODELS_DIR):
    print(f"Le répertoire {MODELS_DIR} n'existe pas.")
else:
    print(f"Répertoire des modèles: {MODELS_DIR}")

if not os.path.exists(TEST_DATA_DIR):
    print(f"Le répertoire {TEST_DATA_DIR} n'existe pas.")
else:
    print(f"Répertoire des données de test: {TEST_DATA_DIR}")

## 2. Analyse des Métriques d'Entraînement

In [ ]:
# Recherche des fichiers de métriques
metrics_files = sorted(glob.glob(os.path.join(MODELS_DIR, "metrics_epoch_*.json")))
print(f"Nombre de fichiers de métriques trouvés: {len(metrics_files)}")

# Chargement des métriques
epochs = []
metrics_data = {
    "FID": [],
    "LPIPS": [],
    "SSIM": [],
    "PSNR": [],
    "Symmetry": [],
    "Wearability": []
}

for metrics_file in metrics_files:
    # Extraction du numéro d'époque
    epoch = int(os.path.basename(metrics_file).split("_")[2].split(".")[0])
    epochs.append(epoch)
    
    # Chargement des métriques
    with open(metrics_file, "r") as f:
        metrics = json.load(f)
    
    # Stockage des métriques
    for key in metrics_data.keys():
        if key in metrics:
            metrics_data[key].append(metrics[key])
        else:
            metrics_data[key].append(None)

# Visualisation des métriques
if len(epochs) > 0:
    fig, axes = plt.subplots(3, 2, figsize=(15, 15))
    axes = axes.flatten()
    
    for i, (key, values) in enumerate(metrics_data.items()):
        # Filtrage des valeurs None
        valid_epochs = [e for e, v in zip(epochs, values) if v is not None]
        valid_values = [v for v in values if v is not None]
        
        if len(valid_values) > 0:
            axes[i].plot(valid_epochs, valid_values, marker='o')
            axes[i].set_title(f"Évolution de {key}")
            axes[i].set_xlabel("Époque")
            axes[i].set_ylabel(key)
            axes[i].grid(True)
    
    plt.tight_layout()
    plt.show()

## 3. Visualisation des Résultats de Génération

In [ ]:
def load_model(model_path):
    """Chargement d'un modèle à partir d'un checkpoint"""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = Hunyuan3DGlassesAdapter().to(device)
    
    # Chargement des poids
    checkpoint = torch.load(model_path, map_location=device)
    if "model_state_dict" in checkpoint:
        model.load_state_dict(checkpoint["model_state_dict"])
    else:
        model.load_state_dict(checkpoint)
    
    model.eval()
    return model

def visualize_3d_model(model_path):
    """Visualisation d'un modèle 3D avec différentes vues"""
    # Chargement du modèle
    mesh = trimesh.load(model_path)
    
    # Création de la scène
    scene = trimesh.Scene(mesh)
    
    # Génération de 4 vues différentes
    angles = [0, 90, 180, 270]
    renders = []
    
    for angle in angles:
        # Rotation de la scène
        rotated_scene = scene.copy()
        rotated_scene.camera_transform = trimesh.transformations.rotation_matrix(
            angle * np.pi / 180, [0, 1, 0]
        )
        
        # Rendu
        render = rotated_scene.save_image(resolution=(512, 512), visible=True)
        renders.append(np.array(Image.open(render)))
    
    # Création de la figure
    fig, axes = plt.subplots(2, 2, figsize=(12, 12))
    axes = axes.flatten()
    
    for i, (render, angle) in enumerate(zip(renders, angles)):
        axes[i].imshow(render)
        axes[i].set_title(f"Vue {angle}°")
        axes[i].axis("off")
    
    plt.tight_layout()
    plt.show()

# Recherche du meilleur modèle
best_model_path = os.path.join(MODELS_DIR, "best_model.pt")
if os.path.exists(best_model_path):
    print(f"Meilleur modèle trouvé: {best_model_path}")
    
    # Chargement du modèle
    try:
        model = load_model(best_model_path)
        print("Modèle chargé avec succès")
    except Exception as e:
        print(f"Erreur lors du chargement du modèle: {e}")
else:
    print(f"Meilleur modèle non trouvé à {best_model_path}")

## 4. Génération de Lunettes 3D à partir d'Images de Test

In [ ]:
# Recherche des images de test
test_images = []
for ext in ["*.jpg", "*.jpeg", "*.png"]:
    test_images.extend(glob.glob(os.path.join(TEST_DATA_DIR, "images", ext)))

print(f"Nombre d'images de test trouvées: {len(test_images)}")

# Génération de lunettes 3D pour quelques images de test
if len(test_images) > 0 and 'model' in locals():
    # Création du générateur
    generator = GlassesGenerator()
    generator.model = model
    
    # Répertoire de sortie pour les modèles générés
    output_dir = "../outputs/generated_samples"
    os.makedirs(output_dir, exist_ok=True)
    
    # Génération pour quelques images
    for i, img_path in enumerate(test_images[:3]):
        print(f"Génération pour {os.path.basename(img_path)}...")
        
        # Nom de fichier de sortie
        base_name = os.path.splitext(os.path.basename(img_path))[0]
        output_path = os.path.join(output_dir, f"{base_name}.glb")
        
        try:
            # Génération du modèle 3D
            generator.generate_glasses(img_path, output_path)
            print(f"Modèle généré: {output_path}")
            
            # Visualisation du modèle
            visualize_3d_model(output_path)
        except Exception as e:
            print(f"Erreur lors de la génération: {e}")

## 5. Comparaison des Résultats entre Différentes Époques

In [ ]:
# Recherche des checkpoints
checkpoints = sorted(glob.glob(os.path.join(MODELS_DIR, "checkpoint_epoch_*.pt")))
print(f"Nombre de checkpoints trouvés: {len(checkpoints)}")

# Sélection de quelques checkpoints pour la comparaison
if len(checkpoints) > 0 and len(test_images) > 0:
    # Sélection des checkpoints à intervalles réguliers
    num_checkpoints = min(4, len(checkpoints))
    selected_checkpoints = [checkpoints[i] for i in np.linspace(0, len(checkpoints)-1, num_checkpoints, dtype=int)]
    
    # Sélection d'une image de test
    test_image = test_images[0]
    print(f"Image de test sélectionnée: {os.path.basename(test_image)}")
    
    # Génération avec chaque checkpoint
    results = []
    
    for checkpoint in selected_checkpoints:
        epoch = int(os.path.basename(checkpoint).split("_")[2].split(".")[0])
        print(f"Génération avec le checkpoint de l'époque {epoch}...")
        
        try:
            # Chargement du modèle
            model = load_model(checkpoint)
            
            # Création du générateur
            generator = GlassesGenerator()
            generator.model = model
            
            # Génération du modèle 3D
            output_path = os.path.join(output_dir, f"comparison_epoch_{epoch}.glb")
            generator.generate_glasses(test_image, output_path)
            
            # Rendu du modèle
            mesh = trimesh.load(output_path)
            scene = trimesh.Scene(mesh)
            render = scene.save_image(resolution=(512, 512), visible=True)
            render = np.array(Image.open(render))
            
            results.append((epoch, render))
        except Exception as e:
            print(f"Erreur: {e}")
    
    # Affichage des résultats
    if len(results) > 0:
        fig, axes = plt.subplots(1, len(results), figsize=(15, 5))
        if len(results) == 1:
            axes = [axes]
        
        for i, (epoch, render) in enumerate(results):
            axes[i].imshow(render)
            axes[i].set_title(f"Époque {epoch}")
            axes[i].axis("off")
        
        plt.tight_layout()
        plt.show()

## 6. Conclusion

Ce notebook nous a permis de visualiser les résultats de l'entraînement du modèle Hunyuan3D adapté pour la génération de lunettes 3D. Nous avons pu observer l'évolution des métriques au cours de l'entraînement et comparer les résultats de génération entre différentes époques.